In [1]:
import numpy as np
import os.path as osp
import cv2
import time
import tqdm
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import keras.backend as K
%matplotlib inline

Using TensorFlow backend.
/home/alex-levashov/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [22]:
%load_ext autoreload
%autoreload 2

from modules.images_viewer import ImagesViewer
from modules.dataset import Dataset
from modules.detector import FCNDetector
from modules.images_viewer import ImagesViewer
from modules.quality import compute_quality, compute_average_precision, find_optimal_threshold, get_precision_recall_curve
import modules.models.loader as loader
import modules.models.simple_model
import config

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
def show(image, size_w=8, size_h=8):
    image = image[:, :, ::-1]
    plt.figure(figsize=(size_w, size_h))
    plt.imshow(image)

# Input

In [4]:
detector = FCNDetector()

In [5]:
dataset = Dataset()

# Show image with rects

In [ ]:
image_data = dataset.images_data[0]
if not config.load_all_images_to_ram:
    image_data.load()
image = image_data.image.copy()
for true_rect in image_data.rects:
    true_rect.draw(image, (0, 255, 0), 5)

mask = cv2.resize(image_data.mask, (0, 0), fx=1.0/config.mask_downsample_rate, fy=1.0/config.mask_downsample_rate)
nms_heat_map = detector.heat_map_nms(mask)
rects = detector.obtain_rects(nms_heat_map, mask)
reduced_rects = FCNDetector.rects_nms(rects)
for rect in reduced_rects:
    rect.draw(image, (255, 0, 0), 3)
print(len(reduced_rects))
show(image, 22, 22)
if not config.load_all_images_to_ram:
    image_data.release()

# Check NMS

In [ ]:
for image_data in tqdm.tqdm_notebook(dataset.images_data):
    if not config.load_all_images_to_ram:
        image_data.load()
    mask = cv2.resize(image_data.mask, (0, 0), fx=1.0/config.mask_downsample_rate, fy=1.0/config.mask_downsample_rate)
    nms_heat_map = detector.heat_map_nms(mask)
    rects = detector.obtain_rects(nms_heat_map, mask)
    reduced_rects = FCNDetector.rects_nms(rects)
    if len(image_data.rects) == len(reduced_rects):
        print("{}:\t ok".format(image_data.image_name))
    else:
        print("{}:\t missmatch {} != {}".format(image_data.image_name, len(image_data.rects), len(reduced_rects)))
        
    if not config.load_all_images_to_ram:
        image_data.release()

# Check Quality

In [ ]:
quality_objects_union = []
for image_data in tqdm.tqdm_notebook(dataset.images_data):
    if not config.load_all_images_to_ram:
        image_data.load()
    mask = cv2.resize(image_data.mask, (0, 0), fx=1.0/config.mask_downsample_rate, fy=1.0/config.mask_downsample_rate)
    nms_heat_map = detector.heat_map_nms(mask)
    rects = detector.obtain_rects(nms_heat_map, mask)
    reduced_rects = FCNDetector.rects_nms(rects)
    quality_objects = compute_quality(image_data.rects, reduced_rects)
    quality_objects_union.extend(quality_objects)
    ap_rate = compute_average_precision(quality_objects)
    th, best_precision, best_recall, best_f1 = find_optimal_threshold(quality_objects)
    
    if ap_rate > 0.9999 and best_precision > 0.9999 and best_recall > 0.9999:
        print("{}:\t ok".format(image_data.image_name))
    else:
        print("{}:\t missmatch {}".format(image_data.image_name, best_recall))
        
    if not config.load_all_images_to_ram:
        image_data.release()

In [ ]:
true_labels = []
predictions = []
for q in quality_objects:
    true_labels.append(q.label)
    predictions.append(q.prediction)
precision, recall, thresholds = precision_recall_curve(true_labels, predictions)

In [ ]:
recall

In [ ]:
find_optimal_threshold(quality_objects_union)

# Estimate Quality

In [6]:
K.clear_session()
fcn_model_module = loader.get_fcn_model_module()
fcn_model = fcn_model_module.FCNModel()
detector = FCNDetector(fcn_model.model, osp.join(fcn_model.weights_dir, 'best_weights.hdf5'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_batch (InputLayer)     (None, None, None, 3)     0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, None, None, 64)    1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, None, None, 64)    36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, None, None, 64)    0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, None, None, 128)   73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, None, None, 128)   147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, None, None, 128)   0         
__________

### One Image Processing

In [54]:
image_data = dataset.images_data[59]
print(image_data.image_name)
image = image_data.image
mask = detector.predict_heat_maps_batch(np.asarray([image]))[0]

61.jpg


In [55]:
mask[:, :, 0].max()

1.1223803

In [56]:
nms_heat_map = detector.heat_map_nms(mask)

In [57]:
rects = detector.obtain_rects(nms_heat_map, mask)
reduced_rects = FCNDetector.rects_nms(rects)

In [58]:
quality_objects = compute_quality(dataset.images_data[0].rects, reduced_rects)

In [59]:
ap_rate = compute_average_precision(quality_objects)
ap_rate

0.48734177215189872

In [60]:
threshold, precision, recall, f1_score = find_optimal_threshold(quality_objects)

/home/alex-levashov/notebook/Keras_Neurons_Detector/src/modules/quality.py:12: RuntimeWarning: invalid value encountered in double_scalars
  return 2*(precision*recall)/(precision + recall)


In [46]:
threshold, precision, recall, f1_score

(0.70637668537139897,
 0.78787878787878785,
 0.33766233766233766,
 0.47272727272727272)

In [47]:
precision, recall, thresholds = get_precision_recall_curve(quality_objects)

In [61]:
filtered_rects = [rect for rect in reduced_rects if rect.score > threshold]
print(len(reduced_rects))
print(len(filtered_rects))

81
81


In [62]:
image_with_rects = image_data.image.copy()
for true_rect in image_data.rects:
    true_rect.draw(image_with_rects, (0, 255, 0), 5)
for rect in reduced_rects:
    rect.draw(image_with_rects, (255, 0, 0), 3)

In [ ]:
images_viewer = ImagesViewer()
images_viewer.set_images([image, image_with_rects, mask*255, nms_heat_map*255])
images_viewer.run()

In [ ]:
cv2.destroyAllWindows()

### Quality over the all images

In [ ]:
def get_quality_objects(image_data):
    if not config.load_all_images_to_ram:
        image_data.load()
    mask = detector.predict_heat_maps_batch(np.asarray([image_data.image]))[0]
    nms_heat_map = detector.heat_map_nms(mask)
    rects = detector.obtain_rects(nms_heat_map, mask)
    reduced_rects = FCNDetector.rects_nms(rects)
    quality_objects = compute_quality(image_data.rects, reduced_rects)
    ap_rate = compute_average_precision(quality_objects)
    
    print("{}:\t {}".format(image_data.image_name, ap_rate))

    if not config.load_all_images_to_ram:
        image_data.release()
        
    return quality_objects

In [ ]:
train_quality_objects = []
for i in tqdm.tqdm_notebook(dataset.train_indices):
    image_data = dataset.images_data[i]
    train_quality_objects.extend(get_quality_objects(image_data))

test_quality_objects = []
for i in tqdm.tqdm_notebook(dataset.test_indices):
    image_data = dataset.images_data[i]
    test_quality_objects.extend(get_quality_objects(image_data))    

train_ap_rate = compute_average_precision(train_quality_objects)
print("Train AP: {}".format(train_ap_rate))
test_ap_rate = compute_average_precision(test_quality_objects)
print("Test AP: {}".format(test_ap_rate))

threshold, best_precision, best_recall, best_f1 = find_optimal_threshold(train_quality_objects)
print("Threshold {}, F1 score {}".format(threshold, f1_score))